# 飞桨常规赛：PALM眼底彩照视盘探测与分割 

本项目使用PaddleSeg进行调用式训练，参考了[https://aistudio.baidu.com/aistudio/projectdetail/2276682]

**声明**

本项目直接fork上述链接生成，但由于九月期间的数据集策略变动，从而对应挂载的数据集无法调用。

## 数据准备

In [1]:
#解压数据
!unzip -oq data/data86770/seg.zip -d /home/aistudio/work

## 引入包

In [42]:
!pip install paddleseg

#导入常用的库
import os
import random
import numpy as np
from random import shuffle
import cv2
import paddle
from PIL import Image
import shutil
import re
from paddle.vision.transforms import functional as F
import os.path
import paddleseg.transforms as T
from paddleseg.datasets import OpticDiscSeg,Dataset
from paddleseg.models import UNet
from paddleseg.models import OCRNet
from paddleseg.models.losses import CrossEntropyLoss,DiceLoss, MixedLoss
from paddleseg.core import train
from paddleseg.core import evaluate
from paddleseg.core import predict
from PIL import Image

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 数据划分

In [43]:
names=[item.split('.')[0] for item in os.listdir('work/seg/Train/fundus_image')]
ori_train_list=[]
img_dir='/home/aistudio/work/seg/Train/fundus_image/'
mask_dir='/home/aistudio/work/seg/Train/masks/'
for name in names:
    ori_train_list.append(img_dir+name+'.jpg '+mask_dir+name+'.png'+'\n')

cut_percent=0.2
cut_point=int(cut_percent*len(ori_train_list))
random.shuffle(ori_train_list)

with open('train.txt','w') as f:
    for item in ori_train_list[cut_point:]:
        f.write(item)

with open('eval.txt','w') as f:
    for item in ori_train_list[:cut_point]:
        f.write(item)

## 训练

In [44]:
train_transforms = [
    T.RandomHorizontalFlip(),# 水平翻转
    T.RandomVerticalFlip(),# 垂直翻转
    T.RandomDistort(0.4),# 随机扭曲
    T.RandomBlur(0.3),# 高斯模糊
    T.RandomScaleAspect(min_scale=0.5,aspect_ratio=0.5),# 随机缩放
    T.Resize(target_size=(512,512)),
    T.Normalize()  # 图像标准化
]

val_transforms = [
    T.Resize(target_size=(512,512)),
    T.Normalize()
]

In [45]:
# 构建训练集
train_dataset = Dataset(
    dataset_root='/home/aistudio',
    train_path='/home/aistudio/train.txt',
    transforms=train_transforms,
    num_classes=2,
    mode='train'
)
#验证集
val_dataset = Dataset(
    dataset_root='/home/aistudio',
    val_path='/home/aistudio/eval.txt',
    transforms=val_transforms,
    num_classes=2,
    mode='val'
)

In [46]:
# unet_model = UNet(num_classes=2,pretrained='/home/aistudio/output/PLAM_model_3/best_model/model.pdparams')#使用预训练模型unet进行训练
unet_model = UNet(num_classes=2)#使用unet进行训练

In [47]:
base_lr =0.001
iters = 1000 

#自动调整学习率
lr =paddle.optimizer.lr.CosineAnnealingDecay(base_lr, T_max=(iters // 3), last_epoch=0.5)
u_optimizer = paddle.optimizer.Adam(lr, parameters=unet_model.parameters())

mixtureLosses = [CrossEntropyLoss(),DiceLoss() ]
mixtureCoef = [0.7,0.3]
losses = {}
losses['types'] = [MixedLoss(mixtureLosses, mixtureCoef)]
losses['coef'] = [1]

In [48]:
#进行训练
train(
    model = unet_model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=u_optimizer,
    save_dir='output/PLAM_model_4',
    iters=iters,  
    batch_size=8, 
    save_interval=200,
    log_iters=20,
    num_workers=2,
    losses=losses,
    use_vdl=False
    )

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/

2021-09-25 09:39:40 [INFO]	[TRAIN] epoch: 1, iter: 20/1000, loss: 0.3025, lr: 0.000991, batch_cost: 2.4208, reader_cost: 1.92957, ips: 3.3047 samples/sec | ETA 00:39:32
2021-09-25 09:40:27 [INFO]	[TRAIN] epoch: 1, iter: 40/1000, loss: 0.1839, lr: 0.000964, batch_cost: 2.3021, reader_cost: 1.81189, ips: 3.4751 samples/sec | ETA 00:36:49
2021-09-25 09:41:11 [INFO]	[TRAIN] epoch: 1, iter: 60/1000, loss: 0.1815, lr: 0.000921, batch_cost: 2.2379, reader_cost: 1.74695, ips: 3.5749 samples/sec | ETA 00:35:03
2021-09-25 09:41:59 [INFO]	[TRAIN] epoch: 1, iter: 80/1000, loss: 0.1842, lr: 0.000863, batch_cost: 2.3997, reader_cost: 1.90834, ips: 3.3337 samples/sec | ETA 00:36:47


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/

2021-09-25 09:42:48 [INFO]	[TRAIN] epoch: 2, iter: 100/1000, loss: 0.1697, lr: 0.000792, batch_cost: 2.4601, reader_cost: 1.96819, ips: 3.2519 samples/sec | ETA 00:36:54
2021-09-25 09:43:34 [INFO]	[TRAIN] epoch: 2, iter: 120/1000, loss: 0.1581, lr: 0.000710, batch_cost: 2.2987, reader_cost: 1.80707, ips: 3.4802 samples/sec | ETA 00:33:42
2021-09-25 09:44:20 [INFO]	[TRAIN] epoch: 2, iter: 140/1000, loss: 0.1610, lr: 0.000621, batch_cost: 2.2714, reader_cost: 1.77917, ips: 3.5220 samples/sec | ETA 00:32:33
2021-09-25 09:45:08 [INFO]	[TRAIN] epoch: 2, iter: 160/1000, loss: 0.1751, lr: 0.000528, batch_cost: 2.3807, reader_cost: 1.88871, ips: 3.3603 samples/sec | ETA 00:33:19


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/

...

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/

2021-09-25 10:18:58 [INFO]	[TRAIN] epoch: 13, iter: 980/1000, loss: 0.0455, lr: 0.000008, batch_cost: 2.4578, reader_cost: 1.96384, ips: 3.2549 samples/sec | ETA 00:00:49
2021-09-25 10:19:48 [INFO]	[TRAIN] epoch: 13, iter: 1000/1000, loss: 0.0434, lr: 0.000000, batch_cost: 2.4917, reader_cost: 1.99884, ips: 3.2106 samples/sec | ETA 00:00:00
2021-09-25 10:19:48 [INFO]	Start evaluating (total_samples: 160, total_iters: 160)...


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
160/160 [==============================] - 28s 175ms/step - batch_cost: 0.1733 - reader cost: 0.159


2021-09-25 10:20:16 [INFO]	[EVAL] #Images: 160 mIoU: 0.8976 Acc: 0.9960 Kappa: 0.8864 
2021-09-25 10:20:16 [INFO]	[EVAL] Class IoU: 
[0.9959 0.7993]
2021-09-25 10:20:16 [INFO]	[EVAL] Class Acc: 
[0.9984 0.8696]
2021-09-25 10:20:18 [INFO]	[EVAL] The model with the best validation mIoU (0.8976) was saved at iter 1000.
<class 'paddle.nn.layer.conv.Conv2D'>'s flops has been counted
<class 'paddle.nn.layer.norm.BatchNorm2D'>'s flops has been counted
Cannot find suitable count function for <class 'paddle.nn.layer.pooling.MaxPool2D'>. Treat it as zero FLOPs.
Total Flops: 124303966208     Total Params: 13404354


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:143: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:238: UserWarning: The dtype of left and right variables are not the same, left dtype is VarType.FP32, but right dtype is VarType.INT32, the right dtype will convert to VarType.FP32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


## 测试

In [49]:
test_list=["work/seg/test/"+item for item in os.listdir("work/seg/test")]

transforms = T.Compose([
    pt.Resize(target_size=(512, 512)),
    pt.Normalize()
])

model = UNet(num_classes=2)

predict(
    model,
    #换自己保存的模型文件
    model_path = 'output/UNet/1000_model/model.pdparams',
    transforms=transforms,
    image_list=image_list,
    save_dir='results',)

2021-09-25 10:22:39 [INFO]	Loading pretrained model from output/UNet/1000_model/model.pdparams
2021-09-25 10:22:39 [INFO]	There are 112/112 variables loaded into UNet.
2021-09-25 10:22:39 [INFO]	Start to predict...


400/400 [==============================] - 110s 275ms/st


## 打包

In [39]:
! mkdir final_result
for img in os.listdir('results/pseudo_color_prediction'):
    img_dir='results/pseudo_color_prediction/'+img
    im=Image.open(img_dir)
    im=Image.fromarray(255-np.array(im)*255)
    im.save('final_result/'+img)
# 打包结果
! zip -q -r result.zip final_result/